In [1]:
from itertools import permutations, combinations

In [2]:
possible_rolls = [
    sum((d1 + 1, d2 + 1, d3 + 1)) 
    for d1 in range(3)
    for d2 in range(3)
    for d3 in range(3)
]

In [5]:
from collections import defaultdict

In [9]:
import pandas as pd
pd.Series(possible_rolls).value_counts()

6    7
7    6
5    6
8    3
4    3
9    1
3    1
dtype: int64

In [74]:
possible_roll_counts = [
    (3, 1),
    (4, 3),
    (5, 6),
    (6, 7),
    (7, 6),
    (8, 3),
    (9, 1),
]

In [116]:
def get_outcome_counts(pos, winner_score=21):
    q = [(pos, 0, 0, 1)]

    results = defaultdict(int)
    while q:
        pos, score, steps, weight = q.pop()
        new_steps = steps + 1
        if score >= winner_score:
            results[steps] += weight
#             print('complete @', 'pos', pos, 'score', score, 'steps', steps, 'weight', weight)
            continue
        for roll, roll_count in possible_roll_counts:
            new_pos = (pos + roll) % 10
            new_score = (new_pos + 1) + score
            new_weight = weight * roll_count
#             print('tick', 'steps', new_steps, 'score', new_score, 'weight', new_weight)
            q.append((new_pos, new_score, new_steps, new_weight))
    return results

In [117]:
get_outcome_counts(0, 4)

defaultdict(int, {1: 27})

In [118]:
get_outcome_counts(0, 5)

defaultdict(int, {1: 26, 2: 27})

In [138]:
p1_results = get_outcome_counts(3)
p2_results = get_outcome_counts(7)
df = pd.DataFrame([dict(p1_results), dict(p2_results)])[range(3, 11)].T
df

,0,1
3,4608,1730
4,249542,230681
5,3219454,5448341
6,24905476,34576828
7,77993473,92929443
8,62172638,57067858
9,8678745,5931126
10,53217,32562


In [141]:
df['wins0'] = df.index.map(lambda i: df.loc[i, 0])
df['wins1'] = df.index.map(lambda i: df.loc[i, 1])

df['loses0'] = df.index.map(lambda i: df.loc[i+1:, 0].sum())
df['loses1'] = df.index.map(lambda i: df.loc[i+1:, 1].sum())

In [150]:
p1_wins = 0
p2_wins = 0
for p1_steps in range(1, 10):
    p1_ways = df.loc[p1_steps, 0] if p1_steps in df.index else 0
    for p2_steps in range(1, 10):
        p2_ways = df.loc[p2_steps, 0] if p2_steps in df.index else 0
        if p1_steps > p2_steps:
            p1_wins += p1_ways
        elif p2_steps > p1_steps:
            p2_wins += p2_ways
p1_wins, p2_wins

(1110762302, 1110762302)

In [113]:
p1_wins = 0
p2_wins = 0
for step in p1_results.keys() | p2_results.keys():
    universes_where_p1_won = sum([p1_results[k] for k in p1_results if k <= step])
    universes_where_p1_lost = sum([p1_results[k] for k in p1_results if k > step])
    
    universes_where_p2_won = sum([p2_results[k] for k in p2_results if k <= step])
    universes_where_p2_lost = sum([p2_results[k] for k in p2_results if k > step])
    p1_wins += universes_where_p1_won * universes_where_p2_lost
    p2_wins += universes_where_p2_won * universes_where_p1_lost
p1_wins, p2_wins

(12854344522627778, 18138371131832112)